# Fraction species affected percentiles (bar-plot)
This notebook, generating a bar-plot of fractions species, follows the same procedure as the notebook for TER percentile. Please see there for detailed explanations. Here, only deviating code will be explained.

The basic setup of the analysis remains the same:

In [ ]:
import observer
import stores
import base
import attrib
import numpy as np
import matplotlib.pyplot as plt

console_observer = observer.ConsoleObserver(print_output=True)
x3df_store = stores.X3dfStore("sample/store", console_observer, "r")
distance_groups_input = base.Input(
    "DistanceGroups", 
    (attrib.Class(np.ndarray), attrib.Scales("space_y/1sqm, space_x/1sqm")), 
    console_observer, 
    base.Output('LandscapeScenario/analysis_distance_groups_values', x3df_store)
)

Unlike in the TER notebook, the endpoint considered here is the fraction species affected whose values are stored under  `"/Fraction_species_affected_SE_dry_weight/Effect"`.

In [ ]:
fraction_species_affected_input = base.Input(
    "FractionSpeciesAffected", 
    (attrib.Class(np.ndarray), attrib.Unit("1"), attrib.Scales("space_y/1sqm, space_x/1sqm, time/day")), 
    console_observer, 
    base.Output('Fraction_species_affected_SE_dry_weight/Effect', x3df_store)
)

Again, only the distance groups of the habitat are loaded:

In [ ]:
distance_groups = distance_groups_input.read(slices=(slice(0, 100), slice(100, 200))).values

The fractions of species affected are loaded for the day of application and the same spatial extent:

In [ ]:
fraction_species_affected = fraction_species_affected_input.read(slices=(slice(0, 100), slice(100, 200), 0)).values

The same distances as in the TER analysis are used:

In [ ]:
distances = [10, 20, 50, 100, float("inf")]

For the analysis of fraction species affected, we are interested in the 90th percentile instead of the 10th percentile.

In [ ]:
quantiles = [np.quantile(fraction_species_affected[distance_groups <= d], 0.9) for d in distances]

The calculated percentiles look as the following:

In [ ]:
quantiles

For preparation of plotting, we define a list of RGB colors in which the individual distance groups should be represented.

In [ ]:
colors = [
    ( 27/255,  57/255,  50/255), 
    ( 45/255, 106/255,  79/255), 
    ( 64/255, 145/255, 108/255), 
    ( 82/255, 183/255, 136/255), 
    (116/255, 198/255, 157/255)
]

Finally, a simple bar-plot visualization using _matplotlib_.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.bar(range((len(distances))), quantiles, color=colors)
ax.set(
    title="90$^{th}$ percentile of fraction species affected - no mitigation",
    ylabel="fraction species affected [-]",
    xlabel="distance group"
)
ax.xaxis.set(ticks=range(len(distances)), ticklabels=distances)
plt.show()